In [1]:
#!/bin/python3
import random
from math import cos, floor
import simpy
import random
import copy
from pprint import pprint
import statistics

from simpy.core import Environment



In [2]:
class Field: 
    def __init__(self):
        self.plans = {}

    def add_entry(self, id, timestep):
        if id not in self.plans:
            self.plans[id] = []
        self.plans[id].append(timestep)
    
    def __str__(self) -> str:
        return self.plans

    def cost(self, timestep, debug=False):
        cost = 1
        timesteps_diff = []
        for (robot_id, timesteps) in self.plans.items():
            for ts in timesteps:
                if ts >= timestep:
                    timesteps_diff.append(ts-timestep)
        timesteps_diff.sort()
        if debug:
            print(timesteps_diff)

        for i in timesteps_diff: 
            if i == 0: 
                continue
            if i == cost:
                cost += 1
            else: 
                break
        return cost

In [3]:
class Grid:     
    def __init__(self, size):
        self.grid = [ [Field() for j in range(size)] for i in range(size) ] 
        self.size = size; 

    def add_path(self, path, id, timestep):   
        """
        [{x: x, y,timestep,id}, ... ,(x,y,timestep,id)]
        """

        for step in path: 
            self.grid[step[0]][step[1]].add_entry(id, timestep)
            timestep += 1

    def shortest_path_iter(self, position, goal, timestep, debug=False):
        explored = {}
        queue     = [ ([position], timestep) ]
        # queue    = self.get_neighboars(position)
        # queue = []

        if position == goal:
            return ( [position], timestep )

        while queue: 
            path = queue.pop(0)
            node = path[0][-1]
            ts = path[1]
        
            
            # cost = self.get_field(node).cost()
            # cur_node  = path[-1]

            # Check if there is already a path going through cur_node which is better
            if node in explored: 
                if ts >= explored[node]:
                    continue     
                    

            for neighbor in self.get_neighboars(node):
                new_path = list(path[0])
                    
                cost = self.get_field(neighbor).cost(ts)
                
                # Add waiting blocks
                for i in range(cost-1):
                    new_path.append(node)

                new_path.append(neighbor)
            
                if neighbor == goal: 
                    return (new_path, ts+cost)

                queue.append( (new_path, ts+cost) )
            
            explored[node] = ts
            
            if debug:
                print("============")
                pprint(path)
                pprint(ts)
                pprint(explored)
                print("============")

    def get_field(self, pos):
        return self.grid[pos[0]][pos[1]]

    def get_neighboars(self, position):
        x = position[0]
        y = position[1]
        
        neighboars = []

        if x > 0:
            neighboars.append( (x-1, y) )

        if x < self.size-1:
            neighboars.append( (x+1, y) )

        if y > 0:
            neighboars.append( (x, y-1) )

        if y < self.size-1:
            neighboars.append( (x, y+1) )

        return neighboars

In [4]:
class Server: 
    def __init__(self, numRobots, gridSize = 16, queue = []):
        self.grid = Grid(gridSize)
        self.gridSize = gridSize
        self.queue = queue
        self.timestep = 0
        self.robots = [ Robot(i) for i in range(numRobots) ]
        self.init_robots()

    def create_path(self, robot, goal, state):
        # assign goal to robot
        # print("[CREATING PATH]", robot.position, "|", goal, end=" -> ")
        (path, score) = self.grid.shortest_path_iter(robot.position, goal, self.timestep)
        # print(path)
        robot.new_goal(path, goal, state)
        self.grid.add_path(path, robot.id, self.timestep)
        return (path, score)

    def init_robots(self):
        taken = []

        if self.gridSize*self.gridSize <= len(self.robots):
            # print("HANUS ER FUCKING NEDERN")
            exit(-1)

        for robot in self.robots: 

            newpos = (random.randint(0, self.gridSize-1),random.randint(0, self.gridSize-1))
            while newpos in taken: 
                newpos = (random.randint(0, self.gridSize-1),random.randint(0, self.gridSize-1))
     
            robot.position = newpos
            taken.append(newpos)

    def handle_deliveries(self):
        for waiting_robot in self.get_robots(status=3):
            # goals = (
            #     0 if self.gridSize - waiting_robot.position[0] > self.gridSize/2 else self.gridSize-1,

            # )
            goal = self.closest_edge(waiting_robot)
            self.create_path(waiting_robot, goal, 2)
            # create_path(robot)

    def closest_edge(self, robot):
        x = robot.position[0]
        y = robot.position[1]
        
        x1 = self.gridSize/2 - x
        y1 = self.gridSize/2 - y
        if abs(x1) > abs(y1): 
            if x1 > 0:
                return (0, y)
            else:
                return (self.gridSize-1, y)
        else:
            if y1 > 0:
                return (x, 0)
            else:
                return (x, self.gridSize-1)

    def handle_orders(self): 
        for idle_robot in self.get_robots(status=0):
            #self.handle_order(robot)
            if len(self.queue) == 0: 
                # no order - do nothing
                return

            order = self.queue.pop(0)
            # print("New order for", idle_robot.id, order)
            self.create_path(idle_robot, order, 1)
    
    def get_robots(self, status = 0):
        robots = []
        for robot in self.robots: 
            if robot.status == status:
                robots.append(robot)
        return robots

    def add_order(self, order):
        self.queue.append(order)

    def run(self, env):
        while True: 
            
            self.handle_orders()
            self.handle_deliveries()

            yield env.timeout(1)
            self.timestep += 1

In [5]:
class Robot:
    def __init__(self, id, position = (-1,-1), goal = (-1,-1), status = 0):
        self.id = id
        self.position = position
        self.goal = goal
        self.path = []

        """
        Status:
        0 = idle (no goal)
        1 = going for picking (towards goal)
        2 = delivering ()
        3 = waiting for deliver
        """
        self.status = status
        #This is the function which needs to run for the simulation to progress:
        # self.action = env.process(self.run())
    
    def move(self, env):
        # try:
        next_position = self.path.pop(0);   
        # print("[", self.id, "]", "moving..", self.position, "->", next_position, "|", env._now)
        self.position = next_position
        # except:
        #     print(self.id, "|", self.position,  "|", self.path, "|", self.goal)
        #     exit(1)

    def run(self, env):
        while True:
            f = open('robot_paths.txt',"a")
            f.write(f"{self.position}\n")
            f.close()
            print(self.position)
            if self.status == 0: 
                yield env.timeout(1)
                continue
            
            if self.status == 1: 
                
                # We have reached the goal
                if (self.position == self.goal):
                    # print(self.id, "We got to the package", self.goal)
                    # We should deliver - and wait for a new path
                    self.status = 3; 
                    yield env.timeout(1)
                    continue
                else: 
                    # Traverse
                    self.move(env)
                    yield env.timeout(1)
                    continue

            if self.status == 2: 
                
                if (self.position == self.goal):
                    # print(self.id, "We deliered to the package", self.goal)
                    self.status = 0
                    yield env.timeout(1)
                    continue
                else:                        
                    # Traverse
                    self.move(env)
                    yield env.timeout(1)
                    continue
                
            if self.status == 3:
                yield env.timeout(1)
                continue      

    def new_goal(self, path, goal, state):
        self.goal = goal
        self.path = path
        self.status = state



In [6]:

def diagnostics(env, server:Server, vars): 
    while True: 

        print(env._now, end="\r")
        vars["queue_size"].append(len(server.queue))
        vars["idle_robots"].append(len(server.get_robots(status=0)))
        vars["active_robots"].append(len(server.get_robots(status=1)))
        vars["delivering_robots"].append(len(server.get_robots(status=2)))
        vars["waiting_robots"].append(len(server.get_robots(status=3)))

        yield env.timeout(1)

    return (queue_size, order_throughput, average_order_sitting_time)
def cleanup(env, server):
    while True:
        for (x, xval) in enumerate(server.grid.grid):
            for (y, field) in enumerate(xval): 
                for (k, plan) in field.plans.items():
                    for (i,ts) in enumerate(plan): 
                        if ts < server.timestep:
                            del plan[i]
        yield env.timeout(65)
#5t -> 25s
#25t -> 25s
#65 -> 23s ---- fundet igennem meget "scientific" metoder
#80 -> 24s 
#110 -> 25s
#155 -> 26s
#200 -> 26s



def send_orders(env, server, max_interval = 10):
    # fifo = []
    # for _i in range(numOrders):
    #     x = random.randint(0,(server.gridSize-1))
    #     y = random.randint(0,(server.gridSize-1))
    #     fifo.append((x,y))

    while True: 
        order = (random.randint(0,(server.gridSize-1)),random.randint(0,(server.gridSize-1)))
        # if(len(fifo) == 0): 
        #     yield env.timeout(1)
        # else: 
        server.add_order(order)
        yield env.timeout(random.randint(0,max_interval))
        
def quality_check(env, server, vars):
    while True: 
        
        occupied = []
        collisons = 0

        for robot in server.robots:
            if not robot.position in occupied:
                occupied.append(robot.position)            
            else:
                collisons += 1

        vars['collisions'].append( collisons )

        yield env.timeout(1)

In [7]:

# seed for consitency in random numbers
random.seed(123)

# Simulation parameters: 

robots = 10
gridSize = 16

# Actual simulation::::
env = simpy.Environment(initial_time=0)
server = Server(robots, gridSize)



diagnostic_variables = {
    "queue_size":[],
    "idle_robots": [], 
    "active_robots": [],
    "delivering_robots": [],
    "waiting_robots": [],
    "collisions": []
}
#######################################
# Add server & robots 
# + start the simulation
#######################################
env.process(server.run(env))
env.process(cleanup(env,server))
env.process(quality_check(env,server, diagnostic_variables))
env.process(send_orders(env, server, max_interval=1))
env.process(diagnostics(env, server, diagnostic_variables))
for robot in server.robots:
    env.process(robot.run(env))
# Run the simluation

until = 200    # set env run time 
env.run(until)

f = open('config_file.txt',"a")
f.truncate(0)
f.write(f"{until},{robots},{gridSize}\n")
f.close()

print("Simulation completed")


(1, 8)
(2, 13)
(8, 3)
(1, 12)
(10, 10)
(1, 5)
(4, 10)
(10, 7)
(5, 0)
(13, 2)
(1, 8)
(2, 13)
(8, 3)
(1, 12)
(10, 10)
(1, 5)
(4, 10)
(10, 7)
(5, 0)
(13, 2)
(1, 8)
(2, 13)
(8, 3)
(1, 12)
(10, 10)
(1, 5)
(4, 10)
(10, 7)
(5, 0)
(13, 2)
(2, 8)
(3, 13)
(7, 3)
(2, 12)
(10, 10)
(1, 5)
(4, 10)
(10, 7)
(5, 0)
(13, 2)
(3, 8)
(4, 13)
(6, 3)
(3, 12)
(11, 10)
(1, 5)
(4, 10)
(10, 7)
(5, 0)
(13, 2)
(4, 8)
(5, 13)
(5, 3)
(4, 12)
(12, 10)
(2, 5)
(4, 10)
(10, 7)
(5, 0)
(13, 2)
(5, 8)
(6, 13)
(4, 3)
(4, 11)
(13, 10)
(3, 5)
(5, 10)
(10, 6)
(5, 0)
(13, 2)
(6, 8)
(7, 13)
(3, 3)
(4, 10)
(13, 11)
(4, 5)
(6, 10)
(10, 5)
(6, 0)
(13, 2)
(7, 8)
(8, 13)
(2, 3)
(4, 9)
(13, 12)
(5, 5)
(7, 10)
(10, 4)
(7, 0)
(12, 2)
(8, 8)
(9, 13)
(1, 3)
(4, 8)
(13, 13)
(6, 5)
(8, 10)
(10, 3)
(8, 0)
(11, 2)
(9, 8)
(10, 13)
(1, 2)
(4, 7)
(13, 14)
(7, 5)
(9, 10)
(10, 2)
(9, 0)
(11, 1)
(10, 8)
(10, 14)
(1, 2)
(4, 6)
(13, 15)
(8, 5)
(10, 10)
(10, 1)
(10, 0)
(11, 1)
(11, 8)
(10, 14)
(1, 2)
(4, 5)
(13, 15)
(9, 5)
(10, 11)
(10, 0)
(11, 0)
(11

(6, 15)
(5, 1)
(8, 15)
(9, 0)
(4, 0)
(10, 8)
(12, 11)
(4, 2)
(14, 7)
(3, 10)
(7, 15)
(5, 2)
(8, 15)
(8, 0)
(4, 0)
(10, 9)
(11, 11)
(5, 2)
(13, 7)
(3, 9)
(8, 15)
(5, 3)
(9, 15)
(7, 0)
(4, 0)
(10, 10)
(11, 11)
(6, 2)
(12, 7)
(3, 8)
(9, 15)
(5, 4)
(10, 15)
(6, 0)
(5, 0)
(10, 11)
(10, 11)
(7, 2)
(11, 7)
(3, 7)
(10, 15)
(5, 5)
(11, 15)
(5, 0)
(6, 0)
(10, 11)
(9, 11)
(8, 2)
(10, 7)
(3, 6)
(11, 15)
(5, 6)
(12, 15)
(4, 0)
(7, 0)
(10, 11)
(8, 11)
(9, 2)
(9, 7)
(3, 5)
(12, 15)
(5, 7)
(12, 14)
(3, 0)
(8, 0)
(10, 12)
(7, 11)
(10, 2)
(8, 7)
(3, 4)
(13, 15)
(5, 8)
(12, 13)
(2, 0)
(9, 0)
(10, 13)
(6, 11)
(11, 2)
(8, 8)
(3, 3)
(14, 15)
(5, 9)
(12, 12)
(2, 1)
(9, 1)
(10, 14)
(5, 11)
(12, 2)
(8, 9)
(3, 2)
(14, 14)
(5, 10)
(12, 11)
(2, 2)
(9, 2)
(10, 15)
(5, 12)
(13, 2)
(8, 10)
(3, 1)
(14, 13)
(5, 11)
(12, 10)
(2, 3)
(9, 3)
(10, 15)
(5, 12)
(13, 3)
(8, 10)
(3, 1)
(14, 12)
(5, 12)
(12, 9)
(2, 4)
(9, 3)
(10, 15)
(5, 12)
(13, 4)
(8, 10)
(3, 1)
(14, 11)
(5, 12)
(12, 8)
(2, 5)
(9, 3)
(10, 14)
(5, 13)
(13, 5)
